In [ ]:
!git lfs clone https://huggingface.co/OpenAssistant/falcon-7b-sft-mix-2000
!pip install git+https://github.com/huggingface/transformers bitsandbytes accelerate torch einops

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [ ]:
# model = "/content/bloom-3b"
# model = "WizardLM/WizardCoder-15B-V1.0"
model = "/content/falcon-7b-sft-mix-2000"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model, local_files_only=True , trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(model, load_in_8bit=False, load_in_4bit=True)

# without quantization
# model = AutoModelForCausalLM.from_pretrained(
#     model,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     local_files_only=True,
#     trust_remote_code=True
# )
# Results of resource consumption -
# SRAM - 3G, VRAM - 14.1G, inference VRAM required <1G


# with 8bit quantization
# model = AutoModelForCausalLM.from_pretrained(
#     model,
#     load_in_8bit=True,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     trust_remote_code=True
# )
# Results of resource consumption -
# SRAM - 4G, VRAM - 9G, inference VRAM required <1G

# # with 4bit quantization
model = AutoModelForCausalLM.from_pretrained(
    model,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
# # Results of resource consumption -
# SRAM - 3.5G, VRAM - 6G, inference VRAM required <1G



# 3B model - non quantized, 8bit quantized and 4bit quantised.
# SRAM - 4G, VRAM - 11.4G, inference VRAM required <1G
# SRAM - 3.5G, VRAM - 4.4G, inference VRAM required <1G
# SRAM - 3.5G, VRAM - 3.4G, inference VRAM required <1G

# 7B model - non quantized, 8bit quantized and 4bit quantised.
# SRAM - 3G, VRAM - 14.1G, inference VRAM required <1G
# SRAM - 4G, VRAM - 9G, inference VRAM required <1G
# SRAM - 3.5G, VRAM - 6G, inference VRAM required <1G

In [ ]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    eos_token_id=tokenizer.eos_token_id,
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1
)

prompt = "<|prompter|>What's the Earth total population<|endoftext|><|assistant|>"

for a in pipeline(prompt):
  print(a)

In [ ]:
prompt = "<|prompter|>What's the Earth total population<|endoftext|><|assistant|>"

for a in pipeline(prompt):
  print(a)